import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=7
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=InlineSkate(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (100, 1883)
first_test_shape= (550, 1883)
classes_quantity= 7
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7}
Count_labels= [ 9 14 18 18 16 14 11]
max(train_feature_Altitude)= 7.0
min(train_feature_Altitude)= -2.2635
first_train_sample=
 [ 2.       -0.036166 -0.15164  ... -0.12786  -0.19202  -0.11038 ]
1 9	2 14	3 18	4 18	5 16	6 14	7 11	

In [3]:
np.shape(ecg)

(100, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 9	2 14	3 18	4 18	5 16	6 14	7 11	
 max = 18


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (9, 481)
Up to class  2 train shape =  (23, 481)
Up to class  3 train shape =  (41, 481)
Up to class  4 train shape =  (59, 481)
Up to class  5 train shape =  (75, 481)
Up to class  6 train shape =  (89, 481)
Up to class  7 train shape =  (100, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0  9 14 18 18 16 14 11]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.7291666652218383
min magnitude class 1  =  -2.508548310896286
after normalizing >>
max magnitude class 1  =  0.733681061164432
min magnitude class 1  =  -1.0787732639697398

 cls 2  >> 
max magnitude class 2  =  3.3943594536308117
min magnitude class 2  =  -2.3324202170418937
after normalizing >>
max magnitude class 2  =  0.6363981969974146
min magnitude class 2  =  -1.027596811261301

 cls 3  >> 
max magnitude class 3  =  3.3222986082221646
min magnitude class 3  =  -2.39417835567284
after normalizing >>
max magnitude class 3  =  0.6154599220948627
min magnitude class 3  =  -1.0455414922706845

 cls 4  >> 
max magnitude class 4  =  3.4087788185571775
min magnitude class 4  =  -2.2646441798184282
after normalizing >>
max magnitude class 4  =  0.6405879428300698
min magnitude class 4  =  -1.0079035466160844

 cls 5  >> 
max magnitude class 5  =  3.600329022216654
min magnitude class 5  =  -2.081825404656493
after normalizing >>
max magnitude clas

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.080262642694062
max tst = 0.7573079028454361


In [14]:
len(wndws_test)

550

In [15]:
np.shape(wndws_test)

(550, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(9, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (100, 480)
y_train =>  (100,)
X_test  =>  (550, 480)
y_test  =>  (550,)
X_valid  =>  (100, 480)
y_valid  =>  (100,)


y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 4s 511ms/step - loss: 2.4978 - accuracy: 0.1400 - val_loss: 2.1204 - val_accuracy: 0.1600
Epoch 2/1000
2/2 [==============================] - 0s 56ms/step - loss: 8.9585 - accuracy: 0.1100 - val_loss: 2.0706 - val_accuracy: 0.1800
Epoch 3/1000
2/2 [==============================] - 0s 41ms/step - loss: 2.0481 - accuracy: 0.1700 - val_loss: 2.0676 - val_accuracy: 0.1400
Epoch 4/1000
2/2 [==============================] - 0s 39ms/step - loss: 2.0053 - accuracy: 0.1700 - val_loss: 2.0529 - val_accuracy: 0.1400
Epoch 5/1000
2/2 [==============================] - 0s 37ms/step - loss: 1.9447 - accuracy: 0.1700 - val_loss: 2.0486 - val_accuracy: 0.1800
Epoch 6/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.8818 - accuracy: 0.2000 - val_loss: 2.0544 - val_accuracy: 0.1800
Epoch 7/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.8975 - accuracy: 0.2600 - val_loss: 2.0389 - val_accuracy: 0.1800
Epoch 8/1000

Epoch 59/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0967 - accuracy: 0.9700 - val_loss: 1.8686 - val_accuracy: 0.2600
Epoch 60/1000
2/2 [==============================] - 0s 37ms/step - loss: 0.0593 - accuracy: 0.9800 - val_loss: 1.8613 - val_accuracy: 0.2200
Epoch 61/1000
2/2 [==============================] - 0s 37ms/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 1.8705 - val_accuracy: 0.2400
Epoch 62/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0568 - accuracy: 0.9900 - val_loss: 1.8379 - val_accuracy: 0.2200
Epoch 63/1000
2/2 [==============================] - 0s 44ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 1.8587 - val_accuracy: 0.2500
Epoch 64/1000
2/2 [==============================] - 0s 40ms/step - loss: 0.0369 - accuracy: 0.9800 - val_loss: 1.8296 - val_accuracy: 0.1900
Epoch 65/1000
2/2 [==============================] - 0s 39ms/step - loss: 0.0202 - accuracy: 0.9900 - val_loss: 1.8592 - val_accuracy: 0.2200
Epoch 

Epoch 116/1000
2/2 [==============================] - 0s 37ms/step - loss: 8.2199e-05 - accuracy: 1.0000 - val_loss: 2.5450 - val_accuracy: 0.2100
Epoch 117/1000
2/2 [==============================] - 0s 36ms/step - loss: 8.1739e-05 - accuracy: 1.0000 - val_loss: 2.5137 - val_accuracy: 0.2300
Epoch 118/1000
2/2 [==============================] - 0s 37ms/step - loss: 8.1267e-05 - accuracy: 1.0000 - val_loss: 2.5002 - val_accuracy: 0.2500
Epoch 119/1000
2/2 [==============================] - 0s 35ms/step - loss: 7.2946e-05 - accuracy: 1.0000 - val_loss: 2.5591 - val_accuracy: 0.2700
Epoch 120/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.1182e-04 - accuracy: 1.0000 - val_loss: 2.6668 - val_accuracy: 0.3000
Epoch 121/1000
2/2 [==============================] - 0s 38ms/step - loss: 2.6975e-04 - accuracy: 1.0000 - val_loss: 2.7381 - val_accuracy: 0.2900
Epoch 122/1000
2/2 [==============================] - 0s 37ms/step - loss: 8.1935e-05 - accuracy: 1.0000 - val_loss: 2

In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

18/18 [==============================] - 0s 8ms/step
array([[ 0,  0, 30,  3,  0, 20,  0],
       [ 0,  0, 47,  9,  0, 30,  0],
       [ 0,  0, 41,  8,  0, 36,  0],
       [ 0,  0, 22,  7,  0, 61,  0],
       [ 0,  0, 37, 14,  0, 50,  0],
       [ 0,  0, 19,  6,  0, 59,  0],
       [ 0,  0,  5,  3,  0, 43,  0]], dtype=int64)
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        53
           2       0.00      0.00      0.00        86
           3       0.20      0.48      0.29        85
           4       0.14      0.08      0.10        90
           5       0.00      0.00      0.00       101
           6       0.20      0.70      0.31        84
           7       0.00      0.00      0.00        51

    accuracy                           0.19       550
   macro avg       0.08      0.18      0.10       550
weighted avg       0.08      0.19      0.11       550



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 2s 425ms/step - loss: 15.0076 - accuracy: 0.0100 - val_loss: 2.0635 - val_accuracy: 0.1800
Epoch 2/1000
2/2 [==============================] - 0s 47ms/step - loss: 2.1236 - accuracy: 0.1800 - val_loss: 2.0674 - val_accuracy: 0.1800
Epoch 3/1000
2/2 [==============================] - 0s 61ms/step - loss: 2.0162 - accuracy: 0.1800 - val_loss: 2.0619 - val_accuracy: 0.1900
Epoch 4/1000
2/2 [==============================] - 0s 43ms/step - loss: 1.9940 - accuracy: 0.2000 - val_loss: 2.0658 - val_accuracy: 0.1800
Epoch 5/1000
2/2 [==============================] - 0s 42ms/step - loss: 1.9562 - accuracy: 0.2300 - val_loss: 2.0185 - val_accuracy: 0.1800
Epoch 6/1000
2/2 [==============================] - 0s 46ms/step - loss: 2.0857 - accuracy: 0.2400 - val_loss: 2.0495 - val_accuracy: 0.1800
Epoch 7/1000
2/2 [==============================] - 0s 43ms/step - loss: 1.7998 - accuracy: 0.3400 - val_loss: 2.0364 - val_accuracy: 0.1800
Epoch 8/100

Epoch 59/1000
2/2 [==============================] - 0s 48ms/step - loss: 0.0676 - accuracy: 0.9900 - val_loss: 2.2217 - val_accuracy: 0.2200
Epoch 60/1000
2/2 [==============================] - 0s 50ms/step - loss: 0.6221 - accuracy: 0.8400 - val_loss: 2.1004 - val_accuracy: 0.1800
Epoch 61/1000
2/2 [==============================] - 0s 47ms/step - loss: 0.5824 - accuracy: 0.7600 - val_loss: 1.9158 - val_accuracy: 0.2700
Epoch 62/1000
2/2 [==============================] - 0s 48ms/step - loss: 1.4354 - accuracy: 0.6500 - val_loss: 1.9695 - val_accuracy: 0.1800
Epoch 63/1000
2/2 [==============================] - 0s 49ms/step - loss: 0.4378 - accuracy: 0.8700 - val_loss: 2.0727 - val_accuracy: 0.2000
Epoch 64/1000
2/2 [==============================] - 0s 45ms/step - loss: 0.0759 - accuracy: 0.9800 - val_loss: 2.5474 - val_accuracy: 0.2200
Epoch 65/1000
2/2 [==============================] - 0s 44ms/step - loss: 0.0714 - accuracy: 0.9800 - val_loss: 2.4048 - val_accuracy: 0.2100
Epoch 

In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

18/18 [==============================] - 0s 8ms/step
array([[ 0, 24, 14, 13,  2,  0,  0],
       [ 0, 52, 14, 14,  2,  0,  4],
       [ 0, 41, 18, 14,  4,  0,  8],
       [ 0, 41,  6, 27,  5,  0, 11],
       [ 0, 38, 19, 28,  6,  0, 10],
       [ 0, 37,  9, 24,  6,  0,  8],
       [ 0, 21,  0, 19,  3,  0,  8]], dtype=int64)
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        53
           2       0.20      0.60      0.31        86
           3       0.23      0.21      0.22        85
           4       0.19      0.30      0.24        90
           5       0.21      0.06      0.09       101
           6       0.00      0.00      0.00        84
           7       0.16      0.16      0.16        51

    accuracy                           0.20       550
   macro avg       0.14      0.19      0.14       550
weighted avg       0.15      0.20      0.15       550



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
